**Training model for dog breed identifiaction android application**

First, let's import everything we'll needed.

In [1]:
from google.colab import files
import zipfile
import tarfile
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from IPython.display import display, HTML
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense
from keras.applications import MobileNetV2, InceptionV3
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import optimizers
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


Then, we import the kaggle.json file, in order to download a dataset in colab.

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"antoinev10","key":"68dbce36bde270219a4043822afb3390"}'}

Once the kaggle.json file is imported, we can install kaggle and download the dataset we want (in this case the stanford-dogs-dataset).

In [3]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

 99% 729M/735M [00:08<00:00, 90.7MB/s]
100% 735M/735M [00:08<00:00, 91.3MB/s]


Now that we have the dataset downloaded as a zip file containing images as a tar file, we will extract the dataset. <br>
We also rename the image directories. Each directory in the images directory is named after a breed, but there are numbers before the breed's name, so we'll rename the directories to keep just the breed's name.

In [0]:
DATASET_FOLDER_PATH = "dataset"
DATASET_ARCHIVE = "stanford-dogs-dataset.zip"
IMAGES_ARCHIVE = os.path.join(DATASET_FOLDER_PATH, "images.tar")
DATA_DIRECTORY = '/content/Images/'

!mkdir -p "dataset"
zip_ref = zipfile.ZipFile(DATASET_ARCHIVE, 'r')
zip_ref.extractall(DATASET_FOLDER_PATH)
zip_ref.close()

tar = tarfile.open(IMAGES_ARCHIVE)
tar.extractall()
tar.close()

for directory_name in os.listdir(DATA_DIRECTORY):
  name = directory_name.split("-")
  
  os.rename(os.path.join(DATA_DIRECTORY, directory_name),
           os.path.join(DATA_DIRECTORY, name[1]))

Our data is now ready to be loaded. We create two arrays X and y, corresponding respectively to the images filenames and the labels. <br>
Then we split our data in a train set and a validation set and create dataframes.

In [5]:
FILENAME_COLUMN = "filename"
CLASS_COLUMN = "class"

X = []
y = []

for directory in os.listdir(DATA_DIRECTORY):
  for image in os.listdir(os.path.join(DATA_DIRECTORY, directory)):
    X.append(os.path.join(directory, image))
    y.append(directory)
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

train_df = pd.DataFrame(
    data={FILENAME_COLUMN: X_train, CLASS_COLUMN:y_train}, 
    columns=[FILENAME_COLUMN, CLASS_COLUMN]
)
valid_df = pd.DataFrame(
    data={FILENAME_COLUMN: X_valid, CLASS_COLUMN:y_valid}, 
    columns=[FILENAME_COLUMN, CLASS_COLUMN]
)

print("train")
display(train_df.head())
print("valid")
display(valid_df.head())

train


,filename,class
0,Kerry_blue_terrier/n02093859_2787.jpg,Kerry_blue_terrier
1,Newfoundland/n02111277_1023.jpg,Newfoundland
2,Irish_wolfhound/n02090721_4326.jpg,Irish_wolfhound
3,Lakeland_terrier/n02095570_563.jpg,Lakeland_terrier
4,Rhodesian_ridgeback/n02087394_6993.jpg,Rhodesian_ridgeback


valid


,filename,class
0,Pekinese/n02086079_11089.jpg,Pekinese
1,Norwich_terrier/n02094258_1778.jpg,Norwich_terrier
2,Staffordshire_bullterrier/n02093256_7577.jpg,Staffordshire_bullterrier
3,Tibetan_terrier/n02097474_6607.jpg,Tibetan_terrier
4,toy_terrier/n02087046_7191.jpg,toy_terrier


We can now preprocess our data and create generators to feed our model later. <br>
We create ```ImageDataGenerator``` instances for the training set and the validation set. When declaring the ```ImageDataGenerator``` instances we define which data augmentation techniques we want to use. <br>
Then we use the ```flow_from_dataframe``` method to create our generators with the dataframes we created earlier.

In [9]:
batch_size = 256
image_size = 224
CLASS_MODE = "categorical"

train_data_generator = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 45,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

validation_data_generator = ImageDataGenerator(
    rescale = 1. / 255
)

train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = DATA_DIRECTORY,
    x_col = FILENAME_COLUMN,
    y_col = CLASS_COLUMN,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = CLASS_MODE
)

validation_generator = validation_data_generator.flow_from_dataframe(
    dataframe = valid_df,
    directory = DATA_DIRECTORY,
    x_col = FILENAME_COLUMN,
    y_col = CLASS_COLUMN,
    shuffle = False,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = CLASS_MODE
)

Found 16464 images belonging to 120 classes.
Found 4116 images belonging to 120 classes.


We can now define our model. We create a checkpoint to save the model after every epoch if it is the best one until now. <br>
Once the model is trained we convert it to a tflite file to use in the android app later.

In [10]:
MODEL_PATH = "model.h5"
K.clear_session()

base_model = MobileNetV2(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(120, activation='softmax')(x)

model = Model(
    inputs = base_model.input,
    outputs = predictions
)

for layer in base_model.layers: layer.trainable = False

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

checkpoint = ModelCheckpoint(
    MODEL_PATH,
    monitor="val_loss",
    verbose=0,
    mode='auto'
)

history = model.fit_generator(
    train_generator,
    train_generator.n // batch_size,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = validation_generator.n // batch_size,
    callbacks=[checkpoint],
    verbose=1
)

def convert_model_to_tflite():
  converter = tf.lite.TFLiteConverter.from_keras_model_file(MODEL_PATH)
  tflite_model = converter.convert()
  open("converted_model.tflite", "wb").write(tflite_model)
  
convert_model_to_tflite()

Epoch 1/10
64/64 [==============================] - 217s 3s/step - loss: 2.3293 - acc: 0.4352 - val_loss: 1.2015 - val_acc: 0.6577
Epoch 2/10
64/64 [==============================] - 196s 3s/step - loss: 1.3117 - acc: 0.6298 - val_loss: 1.2525 - val_acc: 0.6541
Epoch 3/10
64/64 [==============================] - 205s 3s/step - loss: 1.1930 - acc: 0.6541 - val_loss: 1.1115 - val_acc: 0.6915
Epoch 4/10
64/64 [==============================] - 206s 3s/step - loss: 1.0687 - acc: 0.6886 - val_loss: 1.0903 - val_acc: 0.6922
Epoch 5/10
64/64 [==============================] - 205s 3s/step - loss: 1.0287 - acc: 0.6961 - val_loss: 1.0997 - val_acc: 0.6969
Epoch 6/10
64/64 [==============================] - 205s 3s/step - loss: 0.9697 - acc: 0.7169 - val_loss: 1.1415 - val_acc: 0.6870
Epoch 7/10
64/64 [==============================] - 205s 3s/step - loss: 0.9191 - acc: 0.7250 - val_loss: 1.0674 - val_acc: 0.7083
Epoch 8/10
64/64 [==============================] - 206s 3s/step - loss: 0.8771 - a

Finally we create a text file with all the classes. This will also be used in the android app when running the model.

In [0]:
labels = (train_generator.class_indices)

f= open("labels.txt","w+")

for label, index in labels.items():
  f.write(label + "\n")

f.close()